In [1]:
using Laplacians

objc[2490]: Class TKApplication is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[2490]: Class TKMenu is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[2490]: Class TKContentView is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[2490]: Class TKWindow is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.


In [2]:
include("treeSolver.jl")
include("IO.jl")
include("generators.jl")
include("toHighPrecision.jl")
include("verbosePCG.jl")

pcgWorker (generic function with 1 method)

In [3]:
pwd()

"/Users/serbanstan/git/TreePCG/julia"

In [4]:
graphp = "../graphs/xu_1_2/"

"../graphs/xu_1_2/"

In [5]:
a = readFromFile(graphp * "graph.mtx");

In [6]:
tree = readFromFile(graphp * "tree.mtx");

In [9]:
# need to get proper weights for the tree
u,v,w = findnz(tree)
for i in 1:nnz(tree)
    tree[u[i],v[i]] = a[u[i],v[i]]
end

In [10]:
x = readFromFile(graphp * "x.vec");

In [11]:
la = lap(a);
x = x - mean(x)
b = la * x; b = b - mean(b);
a.n

1000

In [13]:
@time F = treeSolver(tree)
@time F(b);

  0.004335 seconds (10.12 k allocations: 710.594 KB)
  0.000545 seconds (13.44 k allocations: 273.750 KB)


In [14]:
# a run on standard Float64 data types
numIts = 1000;
@time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=true);
norm(lap(a) * myx64 - b) / norm(b)
insert!(dbg, 1, "$(numIts) iterations")
insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
writeToFile(graphp * "_log_julia_default.txt", dbg)

Working on iteration 10 with a-norm error 57.56106994311835
Working on iteration 20 with a-norm error 14.925578965357452
Working on iteration 30 with a-norm error 4.282352617771024
Working on iteration 40 with a-norm error 1.114749491872051
Working on iteration 50 with a-norm error 0.2541775307305235
Working on iteration 60 with a-norm error 0.0696332423936379
Working on iteration 70 with a-norm error 0.014494451374467135
Working on iteration 80 with a-norm error 0.003085925812569422
Working on iteration 90 with a-norm error 0.0006640952909330122
Working on iteration 100 with a-norm error 0.00012024362283778763
Working on iteration 110 with a-norm error 2.9149388412790004e-5
Working on iteration 120 with a-norm error 6.777497601237892e-6
Working on iteration 130 with a-norm error 2.0071354868418915e-6
Working on iteration 140 with a-norm error 4.486447746695722e-7
Working on iteration 150 with a-norm error 1.2566283525136805e-7
Working on iteration 160 with a-norm error 3.2794085970331

In [ ]:
numIts = 200
for hp in [64,128,256,512,1024]
    set_bigfloat_precision(hp)
    laHP = toHighPrecision(la; precision=hp)
    treeHP = toHighPrecision(tree; precision=hp)
    xHP = toHighPrecision(x; precision=hp);
    bHP = toHighPrecision(b; precision=hp);
    FHP = treeSolver(treeHP)
    
    @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_$(hp).txt", dbg)
end

Working on iteration 10 with a-norm error 57.56106994311832
Working on iteration 20 with a-norm error 14.92556298755091
Working on iteration 30 with a-norm error 3.7503027818648524
Working on iteration 40 with a-norm error 0.849627323370002
Working on iteration 50 with a-norm error 0.22259934897777614
Working on iteration 60 with a-norm error 0.04707282359966492
Working on iteration 70 with a-norm error 0.008816582422914073
Working on iteration 80 with a-norm error 0.0018440143161407867
Working on iteration 90 with a-norm error 0.00029196334717857084
Working on iteration 100 with a-norm error 7.096039327234141e-5
Working on iteration 110 with a-norm error 1.0728709853897605e-5
Working on iteration 120 with a-norm error 2.3287725656859747e-6
Working on iteration 130 with a-norm error 6.554384314489862e-7
Working on iteration 140 with a-norm error 1.4654770570455712e-7
Working on iteration 150 with a-norm error 3.199838404914416e-8
Working on iteration 160 with a-norm error 7.41081925611